In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key
import shutil

/home/ns1254/miniforge3/envs/robodiff/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


### From existing groups, create more grups

In [2]:
# dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141_b150.hdf5"
# dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141+groups.hdf5"
dataset_path = "/home/ns1254/robomimic/datasets/lift/mh/image_v141+groups.hdf5"

dataset_path = "/home/ns1254/data_robomimic/carl_dataset/mixed_human_original/lift_carl_mix_v4_260.hdf5"

dataset_path = "/home/ns1254/mimicgen/datasets/gen/coffeeprep50.hdf5"

dataset_path = "/home/ns1254/mimicgen/datasets/collected/square134.hdf5"

dataset_path = "/home/ns1254/mimicgen/datasets/collected/square134_2.hdf5"

dataset_path = "/home/ns1254/mimicgen/datasets/collected/coffee_gfs_109.hdf5"

# dataset_path = "/home/ns1254/mimicgen/datasets/collected/kitchen82.hdf5"
# dataset_path = "/home/ns1254/mimicgen/datasets/collected/kitchen82_31.hdf5"

dataset_path = "/home/ns1254/dataset_mimicgen/mug124.hdf5"

# # dataset_path = "/home/ns1254/dataset_mimicgen/sawyer_coffee_gfs76.hdf5" 
# # dataset_path = "/home/ns1254/dataset_mimicgen/coffee_gfs_109.hdf5"

# dataset_path = "/home/ns1254/dataset_mimicgen/coffee_gfs_109_2.hdf5"
# dataset_path = "/home/ns1254/dataset_mimicgen/square134_2.hdf5"

dataset_path = "/home/ns1254/dataset_mimicgen/kitchen_e20_lay20.hdf5"

# dataset_path = "/home/ns1254/dataset_mimicgen/coffee_gfs_109_2_sub.hdf5"

In [2]:
dataset_path="/home/ns1254/nn_project/data/pusht/pusht_v7_zarr_206.hdf5"

In [3]:
f = h5py.File(dataset_path, "r+")
demos = list(f["data"].keys())

In [4]:
lengths=[]
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  206
Max length:  246
Min length:  49
Mean length:  124.51456310679612


In [17]:
for key in f['mask'].keys():
    print(key, f['mask'][key])  

all <HDF5 dataset "all": shape (206,), type "|S8">
f100 <HDF5 dataset "f100": shape (100,), type "|S8">
f150 <HDF5 dataset "f150": shape (150,), type "|S8">
f200 <HDF5 dataset "f200": shape (200,), type "|S8">
f50 <HDF5 dataset "f50": shape (50,), type "|S8">


In [9]:
# del f['mask']['g40']

In [18]:
f.close()

In [6]:
good_demos=[demo.decode('utf-8') for demo in f['mask']['g20']]
good_demos.sort(key=lambda x: int(x.split('_')[-1]))
bad_demos=[demo.decode('utf-8') for demo in f['mask']['lay20']] 
len(good_demos), len(bad_demos) 

(20, 20)

In [8]:
good_demos=[demo.decode('utf-8') for demo in f['mask']['g40']]
good_demos.sort(key=lambda x: int(x.split('_')[-1]))
bad_demos=[demo.decode('utf-8') for demo in f['mask']['bad_first']]
bad_demos2=[demo.decode('utf-8') for demo in f['mask']['bad_second']] 
bad_demos3=[demo.decode('utf-8') for demo in f['mask']['bad_third']]
bad_demos4=[demo.decode('utf-8') for demo in f['mask']['bad_st']]
len(good_demos), len(bad_demos), len(bad_demos2) , len(bad_demos3), len(bad_demos4)

(40, 16, 16, 22, 10)

In [15]:
# goodbad=good_demos[:10]+bad_demos 
goodbad=demos 
goodbad=np.array(goodbad, dtype='S8')
goodbad.shape

(206,)

In [16]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in goodbad])
filter_name='all'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [12]:
good

array([b'demo_1', b'demo_2', b'demo_3', b'demo_4', b'demo_10', b'demo_11',
       b'demo_12', b'demo_13', b'demo_14', b'demo_15', b'demo_16',
       b'demo_17', b'demo_18', b'demo_19', b'demo_20', b'demo_21',
       b'demo_22', b'demo_23', b'demo_24', b'demo_25', b'demo_26',
       b'demo_27', b'demo_28', b'demo_29', b'demo_30', b'demo_31',
       b'demo_32', b'demo_33', b'demo_34', b'demo_35', b'demo_36',
       b'demo_37', b'demo_38', b'demo_39', b'demo_40', b'demo_41',
       b'demo_42', b'demo_43', b'demo_44', b'demo_45'], dtype='|S8')

In [10]:
goodbad = good_demos[:30] + bad_demos[:10] + bad_demos2[:10]  
goodbad=np.array(goodbad, dtype='S8')
goodbad.shape

(50,)

In [11]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in goodbad])
filter_name='g30b20'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [6]:
good_demos=[demo.decode('utf-8') for demo in f['mask']['better_operator_1']]
bad_demos=[demo.decode('utf-8') for demo in f['mask']['worse_operator_1']]
len(good_demos) , len(bad_demos)

(50, 50)

In [7]:
good_demos[:5], bad_demos[:5]

(['demo_0', 'demo_1', 'demo_10', 'demo_11', 'demo_12'],
 ['demo_200', 'demo_201', 'demo_202', 'demo_203', 'demo_204'])

In [8]:
goodbad = good_demos + bad_demos
 
goodbad=np.array(goodbad, dtype='S8')
goodbad.shape

(100,)

In [9]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in goodbad])
filter_name='w1b1'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [10]:
g80 = good_demos[:80] 
 
g80=np.array(g80, dtype='S8')
g80.shape

(80,)

In [11]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in g80])
filter_name='g80'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

filter_keys=sorted([elem for elem in goodbad])
filter_name='p20i'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [27]:
f.close()

In [9]:
# good_demos=[demo.decode('utf-8') for demo in f['mask']['expert']] 
good_demos=[demo.decode('utf-8') for demo in f['mask']['good']] 
bad_hover=[demo.decode('utf-8') for demo in f['mask']['bad_hover']]
bad_touch=[demo.decode('utf-8') for demo in f['mask']['bad_touch']]
bad_intentional=[demo.decode('utf-8') for demo in f['mask']['intentional']]
bad_other=[demo.decode('utf-8') for demo in f['mask']['other_task']]
try_good = [demo.decode('utf-8') for demo in f['mask']['try_good']]
len(good_demos), len(bad_hover), len(bad_touch), len(bad_intentional), len(bad_other), len(try_good)

(100, 20, 20, 40, 40, 20)

In [10]:
p20base=good_demos[:80]
p20h=p20base + bad_hover[:20]
p20t=p20base + bad_touch[:20]
p20i=p20base + bad_intentional[:20]
p20o=p20base + bad_other[:20]
p20tg=p20base + try_good[:20]
p20bad=p20base + bad_hover[:10] + bad_touch[:10]
 

p20base=np.array(p20base, dtype='S8')
p20h=np.array(p20h, dtype='S8')
p20t=np.array(p20t, dtype='S8')
p20i=np.array(p20i, dtype='S8')
p20o=np.array(p20o, dtype='S8')
p20tg=np.array(p20tg, dtype='S8')
p20bad=np.array(p20bad, dtype='S8')

p20base.shape, p20h.shape, p20t.shape, p20i.shape, p20o.shape, p20tg.shape, p20bad.shape

((80,), (100,), (100,), (100,), (100,), (100,), (100,))

In [11]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in p20i])
filter_name='p20i2'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [7]:
new_demos=[name.decode('utf-8') for name in f['mask']['new_demos']]
short_demos=[]
for demo_name in new_demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    if num_samples<700:
        short_demos.append(demo_name)

len(short_demos)

25

In [8]:
g100b25= good_demos  + short_demos
 
g100b25=np.array(g100b25, dtype='S8')
g100b25.shape

(125,)

In [9]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in g100b25])
filter_name='g100b25'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [13]:
f.close()